In [1]:

from time import time
import numpy as np
from tst.domutil.util import *
from tst.domutil.test import *


from copy import copy
from scipy.sparse import *
from scipy.spatial.distance import *


def Dnorm(hc1, hc2, hcboth):
    log1 =  np.log10( hc1 + 1) 
    log2 =  np.log10( hc1 + 1) 
    log3 =  np.log10( hcboth + 1) 
    norm = (log1 + log2) / 2 - log3 
    return norm

reset_database_connection()

# import sys
reload(sys.modules["tst.domutil.util"])
from tst.domutil.util import *

import sys
reload(sys.modules["tst.domutil.test"])
from tst.domutil.test import *


In [2]:
%%time
reset_database_connection()

sDB = seqDB.objects.get(name = 'CATH')
# sDB.sequence_set.count()
seq = sDB.sequence_set.first()

# seqset_all = sDB.sequence_set.all().prefetch_related('hmmprofile_set').filter(hit4hmm2hsp__bitscore__gte=30)

# seq.HMMprofile
# seq.hmmprofile
# dir(seq)
# seq.hmmprofile_set.count()


CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 14.3 ms


# Counting HMM-HMM intermediate sequence
## For Homsf nodes
## (sequence wise, Parallel)


In [90]:
%%time
# from tst.utils_db import *

seqs = sDB.sequence_set.all().defer('hmmprofile__text')#[:1000]
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'H')
# matObj.force = 1
matObj.hit_sum()
matObj.MySQL_hcount()

D_raw = matObj.Draw_para(pcount = 7)
D_norm = matObj.Dnorm( D_raw )

pk_dump(('D_raw',D_raw),  matObj.letter + '_410')
pk_dump( ('D_norm',D_norm), matObj.letter + '_410')
pk_dump( ('hcounts',matObj.hcounts), matObj.letter + '_410')

KeyboardInterrupt: 

Process PoolWorker-95:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-91:
    self.run()
Process PoolWorker-94:
Process PoolWorker-92:
Process PoolWorker-93:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self._target(*self._args, **self._kwargs)
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 102, in worker


In [83]:
matObjH = matObj

# Counting HMM-HMM intermediate sequence
## For S35 nodes
## (sequence wise, Parallel)


In [84]:
%%time
# from tst.utils_db import *

seqs = sDB.sequence_set.all().defer('hmmprofile__text')#[:1000]
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'S')
matObj.forward_dict;
matObj.reverse_dict;
# matObj.force = 1 
# matObj.field
matObj.hit_sum()
matObj.MySQL_hcount()

D_raw = matObj.Draw_para(pcount = 7)
D_norm = matObj.Dnorm( D_raw )

pk_dump(('D_raw',D_raw),  matObj.letter + '_410')
pk_dump( ('D_norm',D_norm), matObj.letter + '_410')
pk_dump( ('hcounts',matObj.hcounts), matObj.letter + '_410')

0
[Worker]0 of 80407
[Worker]600 of 80407
[Listener]0 of -1
1
[Listener]600 of -1
[Worker]1200 of 80407
2
[Worker]1800 of 80407
[Listener]1200 of -1
3
[Worker]2400 of 80407
[Listener]1800 of -1
4
[Worker]3000 of 80407
[Listener]2400 of -1
5
[Worker]3600 of 80407
[Listener]3000 of -1
6
[Worker]4200 of 80407
[Listener]3600 of -1
7
[Worker]4800 of 80407
[Listener]4200 of -1
8
[Worker]5400 of 80407
[Listener]4800 of -1
9
[Worker]6000 of 80407
[Listener]5400 of -1
10
[Worker]6600 of 80407
[Listener]6000 of -1
11
[Worker]7200 of 80407
[Listener]6600 of -1
12
[Worker]7800 of 80407
[Listener]7200 of -1
13
[Worker]8400 of 80407
[Listener]7800 of -1
14
[Worker]9000 of 80407
 [Worker]9000 of 80407
[Listener]8400 of -1
15
 [Worker]9600 of 80407
[Worker]9600 of 80407
[Listener]9000 of -1
16
[Worker]10200 of 80407
[Listener]9600 of -1
17
[Worker]10800 of 80407
[Listener]10200 of -1
18
[Worker]11400 of 80407
[Listener]10800 of -1
19
[Worker]12000 of 80407
[Listener]11400 of -1
20
[Worker]12600 of 804

86000 of 1234806
87000 of 1234806
88000 of 1234806
89000 of 1234806
90000 of 1234806
91000 of 1234806
92000 of 1234806
93000 of 1234806
94000 of 1234806
95000 of 1234806
96000 of 1234806
97000 of 1234806
98000 of 1234806
99000 of 1234806
100000 of 1234806
101000 of 1234806
102000 of 1234806
103000 of 1234806
104000 of 1234806
105000 of 1234806
106000 of 1234806
107000 of 1234806
108000 of 1234806
109000 of 1234806
110000 of 1234806
111000 of 1234806
112000 of 1234806
113000 of 1234806
114000 of 1234806
115000 of 1234806
116000 of 1234806
117000 of 1234806
118000 of 1234806
119000 of 1234806
120000 of 1234806
121000 of 1234806
122000 of 1234806
123000 of 1234806
124000 of 1234806
125000 of 1234806
126000 of 1234806
127000 of 1234806
128000 of 1234806
129000 of 1234806
130000 of 1234806
131000 of 1234806
132000 of 1234806
133000 of 1234806
134000 of 1234806
135000 of 1234806
136000 of 1234806
137000 of 1234806
138000 of 1234806
139000 of 1234806
140000 of 1234806
141000 of 1234806
142000

550000 of 1234806
551000 of 1234806
552000 of 1234806
553000 of 1234806
554000 of 1234806
555000 of 1234806
556000 of 1234806
557000 of 1234806
558000 of 1234806
559000 of 1234806
560000 of 1234806
561000 of 1234806
562000 of 1234806
563000 of 1234806
564000 of 1234806
565000 of 1234806
566000 of 1234806
567000 of 1234806
568000 of 1234806
569000 of 1234806
570000 of 1234806
571000 of 1234806
572000 of 1234806
573000 of 1234806
574000 of 1234806
575000 of 1234806
576000 of 1234806
577000 of 1234806
578000 of 1234806
579000 of 1234806
580000 of 1234806
581000 of 1234806
582000 of 1234806
583000 of 1234806
584000 of 1234806
585000 of 1234806
586000 of 1234806
587000 of 1234806
588000 of 1234806
589000 of 1234806
590000 of 1234806
591000 of 1234806
592000 of 1234806
593000 of 1234806
594000 of 1234806
595000 of 1234806
596000 of 1234806
597000 of 1234806
598000 of 1234806
599000 of 1234806
600000 of 1234806
601000 of 1234806
602000 of 1234806
603000 of 1234806
604000 of 1234806
605000 of 

1016000 of 1234806
1017000 of 1234806
1018000 of 1234806
1019000 of 1234806
1020000 of 1234806
1021000 of 1234806
1022000 of 1234806
1023000 of 1234806
1024000 of 1234806
1025000 of 1234806
1026000 of 1234806
1027000 of 1234806
1028000 of 1234806
1029000 of 1234806
1030000 of 1234806
1031000 of 1234806
1032000 of 1234806
1033000 of 1234806
1034000 of 1234806
1035000 of 1234806
1036000 of 1234806
1037000 of 1234806
1038000 of 1234806
1039000 of 1234806
1040000 of 1234806
1041000 of 1234806
1042000 of 1234806
1043000 of 1234806
1044000 of 1234806
1045000 of 1234806
1046000 of 1234806
1047000 of 1234806
1048000 of 1234806
1049000 of 1234806
1050000 of 1234806
1051000 of 1234806
1052000 of 1234806
1053000 of 1234806
1054000 of 1234806
1055000 of 1234806
1056000 of 1234806
1057000 of 1234806
1058000 of 1234806
1059000 of 1234806
1060000 of 1234806
1061000 of 1234806
1062000 of 1234806
1063000 of 1234806
1064000 of 1234806
1065000 of 1234806
1066000 of 1234806
1067000 of 1234806
1068000 of 1

# Testing Case that ignores all exceptions
## For S35 nodes
## (sequence wise, Parallel)


In [87]:
%%time
reload(sys.modules["tst.utils_db"])
from tst.utils_db import *

seqs = sDB.sequence_set.all().defer('hmmprofile__text')[:1000]
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'S')
matObj.force = 1
matObj.hit_sum()
matObj.MySQL_hcount()

D_raw = matObj.Draw_para(pcount = 7)
D_norm = matObj.Dnorm( D_raw )

pk_dump( ('D_raw' , D_raw  ),  'test_' + matObj.letter + '_410')
pk_dump( ('D_norm', D_norm ),  'test_' + matObj.letter + '_410')
pk_dump( ('hcounts',matObj.hcounts), matObj.letter + '_410')

settings.USE_MODELLER
CPU times: user 9.34 s, sys: 352 ms, total: 9.69 s
Wall time: 39.5 s


In [98]:
%%time
reload(sys.modules["tst.utils_db"])
from tst.utils_db import *

seqs = sDB.sequence_set.all().defer('hmmprofile__text')
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'S')
# matObj.force = 1
matObj.hit_sum()
pk_dump( ('hcounts',matObj.hcounts), matObj.letter + '_410')

settings.USE_MODELLER
CPU times: user 31 s, sys: 796 ms, total: 31.8 s
Wall time: 1min 12s


In [ ]:
from tst.utils_db import *
reload(sys.modules["tst.utils_db"])
from tst.utils_db import *

In [55]:
print matObj.D_both

None


----
----
----
----
### TESTING ONGOING!!!!!!!!!!!!!!!!!!!!!!!!
----
----
----
----

In [7]:
%%time
seqs = sDB.sequence_set.all().defer('hmmprofile__text')
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'H')
matObj.load('H_410')
D_geoavg = matObj.Dgeoavg()
%time matObj.Dboth()

%time matObj.to_MySQL()

[Msg] managed to load D_raw from D_raw/H_410
[Msg] managed to load D_norm from D_norm/H_410
[Msg] managed to load hcounts from hcounts/H_410
[Msg] failed  to load D_both from D_both/H_410
0 of 2931
1000 of 2931
2000 of 2931
0 of -1

[SUMMARY]:
	[Task]: finshed zipping sparse matrix from  
	[Failrate]0 instances of 2931 failed, ( 0.00% )
	[Duration]:Ended after 0.1190 sec
0 of -1

[SUMMARY]:
	[Task]: finshed zipping sparse matrix from  
	[Failrate]0 instances of 2931 failed, ( 0.00% )
	[Duration]:Ended after 0.1980 sec
CPU times: user 452 ms, sys: 24 ms, total: 476 ms
Wall time: 461 ms
inserting ISS hits between S35 0 of -1

[SUMMARY]:inserting ISS hits between S35 
	[Task]: finshed  from  
	[Failrate]0 instances of 2931 failed, ( 0.00% )
	[Duration]:Ended after 25.6545 sec
CPU times: user 20.4 s, sys: 692 ms, total: 21.1 s
Wall time: 25.7 s
CPU times: user 21 s, sys: 716 ms, total: 21.7 s
Wall time: 26.4 s


In [46]:
it = D_geoavg.iteritems()
(x,y),z = next(it)
vx = matObj.hcounts[x]
vy = matObj.hcounts[y]
print vx,vy,(vx*vy)**0.5
print x,y,z
print next(it)
print next(it)
print next(it)

35 167 76.4525996942
1404 1413 76.4525996942
((182, 1123), 212.48529360875779)
((1616, 1744), 205.23157651784484)
((1575, 2176), 502.30468841132671)


In [126]:
reload(sys.modules["tst.utils_db"])
from tst.utils_db import *

seqs = sDB.sequence_set.all().defer('hmmprofile__text')
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'S')
matObj.load('S_410')
# matObj.D_both = wrap_mat( matObj.D_raw,matObj.D_norm)
matObj.to_MySQL()

settings.USE_MODELLER
[Msg] managed to load D_raw from D_raw/S_410
[Msg] managed to load D_norm from D_norm/S_410
[Msg] managed to load hcounts from hcounts/S_410
[Msg] failed  to load D_both from D_both/S_410
0 of -1
10000 of -1
20000 of -1
30000 of -1
40000 of -1
50000 of -1
60000 of -1
70000 of -1
80000 of -1
90000 of -1
100000 of -1
110000 of -1
120000 of -1
130000 of -1
140000 of -1
150000 of -1
160000 of -1
170000 of -1
180000 of -1
190000 of -1
200000 of -1
210000 of -1
220000 of -1
230000 of -1
240000 of -1
250000 of -1
260000 of -1
270000 of -1
280000 of -1
290000 of -1
300000 of -1
310000 of -1
320000 of -1
330000 of -1
340000 of -1
350000 of -1
360000 of -1
370000 of -1
380000 of -1
390000 of -1
400000 of -1
410000 of -1
420000 of -1
430000 of -1
440000 of -1
450000 of -1
460000 of -1
470000 of -1
480000 of -1
490000 of -1
500000 of -1
510000 of -1
520000 of -1
530000 of -1
540000 of -1
550000 of -1
560000 of -1
570000 of -1
580000 of -1
590000 of -1
600000 of -1
610000 of -

In [125]:
reload(sys.modules["tst.utils_db"])
from tst.utils_db import *

seqs = sDB.sequence_set.all().defer('hmmprofile__text')
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'H')
matObj.load('H_410')
# matObj.D_both = wrap_mat( matObj.D_raw,matObj.D_norm)
matObj.to_MySQL()

settings.USE_MODELLER
[Msg] managed to load D_raw from D_raw/H_410
[Msg] managed to load D_norm from D_norm/H_410
[Msg] managed to load hcounts from hcounts/H_410
[Msg] failed  to load D_both from D_both/H_410
0 of -1

[SUMMARY]:
	[Task]: finshed zipping sparse matrix from  
	[Failrate]0 instances of 2931 failed, ( 0.00% )
	[Duration]:Ended after 0.1349 sec
inserting ISS hits between S35 0 of -1

[SUMMARY]:inserting ISS hits between S35 
	[Task]: finshed  from  
	[Failrate]0 instances of 2931 failed, ( 0.00% )
	[Duration]:Ended after 1.4490 sec


In [173]:
%%time
nodes = classification.objects.filter( pk__in = matObj.reverse_dict )
nodes.count()
l = nodes.values_list(matObj.rfield);

CPU times: user 184 ms, sys: 16 ms, total: 200 ms
Wall time: 260 ms


In [184]:
%%time
l = l.annotate(hcount = Count(matObj.rfield))
print l[0]

(13031L, 7)
CPU times: user 36 ms, sys: 4 ms, total: 40 ms
Wall time: 24.4 s


In [38]:
"hit4hmm2hsp__query__%s__in" % matObj.field

'hit4hmm2hsp__query__cath_node__parent__id__in'

In [4]:
%%time
# from tst.utils_db import *
seqs = sDB.sequence_set.all().defer('hmmprofile__text')
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'H')
matObj.hit_sum()

CPU times: user 84 ms, sys: 8 ms, total: 92 ms
Wall time: 28.5 s


In [5]:
matObj.MySQL_hcount()


[SUMMARY]:inserting hit_summary for CATH nodes 
	[Task]: finshed  from  
	[Failrate]0 instances of 0 failed, ( 0.00% )
	[Duration]:Ended after 10.0320 sec


In [6]:
%%time
# from tst.utils_db import *
seqs = sDB.sequence_set.all().defer('hmmprofile__text')
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'S')
matObj.hit_sum()
matObj.MySQL_hcount()

inserting hit_summary for CATH nodes 0 of -1
inserting hit_summary for CATH nodes 10000 of -1
inserting hit_summary for CATH nodes 20000 of -1

[SUMMARY]:inserting hit_summary for CATH nodes 
	[Task]: finshed  from  
	[Failrate]0 instances of 21086 failed, ( 0.00% )
	[Duration]:Ended after 39.6254 sec
CPU times: user 31.7 s, sys: 848 ms, total: 32.6 s
Wall time: 1min 3s


In [79]:
i = 309887; out = 312

# i = 282067; out = 25

i = matObj.forward_dict[i]
hc = matObj.hcounts[i]
assert hc == out,"exp: %d, actual: %d" % (out,hc)

In [186]:
############################################################################
####### For some reason this does not work #################################
############################################################################
############################################################################

seqs = sDB.sequence_set.all().defer('hmmprofile__text')
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'S')
nodes = classification.objects.filter( pk__in = matObj.reverse_dict )

%time l = l.annotate(hcount = Count(matObj.rfield)); print l[0]
%time hcounts = dict(l.values_list('id','hcount'))


(13031L, 7)
CPU times: user 28 ms, sys: 12 ms, total: 40 ms
Wall time: 23.6 s
CPU times: user 64 ms, sys: 4 ms, total: 68 ms
Wall time: 23.6 s


In [189]:
%%time
l = nodes.defer('hmmprofile__text').values_list(matObj.rfield).annotate(hcount = Count(matObj.rfield))

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.26 ms


In [204]:
%%time
l = nodes.values_list("id", matObj.rfield).distinct();
hcounts1 = Counter(x for x,y in l)

CPU times: user 504 ms, sys: 20 ms, total: 524 ms
Wall time: 24.1 s


In [3]:
# from tst.utils_db import *

def draft_hsummary(node, sDB, hcount, node_id = None):
    if node_id:
        node = classifictaion.objects.get(pk = node_id)
    qset = node.hit_summary_set.filter(seqDB=sDB)
    if qset.exists():
        q = qset[0]
        q.hcount = hcount
        q.save()
        return []
    else:
        jdict = {"node": node,
                "seqDB":sDB,
                "hcount":hcount,
                }
        return hit_summary(**jdict)
        # q = hit_summary()
        
class ctmat(object):
    def __init__(self, hmms, sDB = None, seqs = None, letter = None, 
        alias = None,
        force = 0,):
        self.force = force 
        if sDB:
            seqs = sDB.sequence_set.all().defer('hmmprofile__text')
            self.sDB = sDB
        else:
            self.sDB = seqs[0].seqDB            
        self.seqs = seqs
        self.hmms = hmms
        self.D_raw = None
        self.D_norm = None
        self.D_both = None
        self.hits = None
        
        if letter:
            self.set_letter(letter)
        self.alias = alias or 'test'
        self.name = '%s_%s' % (self.alias, self.letter)
        pass
    def set_letter(self, letter = 'H'):
        self.letter = letter
        self.field = forward_field[letter]
        self.rfield = reverse_field[letter]
        self.reverse_dict = sorted(set( self.hmms.values_list( self.field, flat = True)))
        self.forward_dict = list2dict( self.reverse_dict )
        self.l = len(self.reverse_dict)
        self.hits = self.seqs.filter(**{"hit4hmm2hsp__query__%s__in" % self.field : self.reverse_dict })

    
    def OOLD_hit_sum(self,):
        all_nodes = classification.objects.filter(version__name='v4_1_0')
        curr_nodes = all_nodes.filter(level__letter = self.letter).defer(self.rfield.replace("__hits","__text"))
        
        #   %%time
        ########################################################
        ###### Slightly faster    ####################
        ########################################################
        qset = curr_nodes.annotate(
        hcount=Count(Case(
                When(**{
                    self.rfield+"__seqDB":self.sDB,
                    "then":1}),
        #         output_field=CharField(),
            ))
        )
        hcounts = qset.values_list('id','hcount')
        # dict()
        # lst = list(hcounts1)
        d = dict( list(hcounts) )
        hcounts = [ d[x] for x in self.reverse_dict]
        self.hcounts = hcounts
    
    def hit_sum(self,):
        if isinstance(self.hits, type(None)):
            self.hits = self.seqs.filter(**{"hit4hmm2hsp__query__%s__in" % self.field : self.reverse_dict })
        qset = self.hits.values_list("hit4hmm2hsp__query__%s" % self.field, ).annotate(hcount = Count("id",distinct = True))
        hcounts0 = dict( qset.values_list("hit4hmm2hsp__query__%s" % self.field,"hcount"))
        ex = set(self.reverse_dict) and set(hcounts0.keys())
        self.hcounts = [ hcounts0[i] if i in ex else 0 for i in self.reverse_dict ]
        return (hcounts0)

    def OLD_hit_sum(self,):        
        reset_database_connection()
        hcounts = []
        all_nodes = classification.objects.filter(version__name='v4_1_0')
        curr_nodes = all_nodes.filter(level__letter = self.letter).defer(self.rfield.replace("__hits","__text"))
        sids = set(self.sDB.sequence_set.values_list('id',flat = True))

        for node_id in self.reverse_dict:
            qset = curr_nodes.filter(id = node_id)
            hlist = set( qset.values_list(self.rfield, flat =  True) )
            hcount = len( hlist & sids)
            hcounts.append(hcount)
        self.hcounts = hcounts

        
        
    def Draw_para(self,  bsize = 600, pcount = 5):
        class MyManager(SyncManager): pass
        MyManager.register('counter',counter)

        
        field = self.field
        self.seqs = self.seqs.prefetch_related('hmmprofile_set__' + self.field.rstrip('__id') ) 
        
        ##### Actually firing subprocesses #####

        reset_database_connection()
        INPUT = self.seqs
        INPUTs = batch_qs( INPUT, bsize )
        l = self.l
        OUTPUT = dok_matrix( (l + 1, l + 1), dtype = 'int')

        local_listener = listener      
        local_worker = batch_worker

        if __name__=='__main__':
                global m
                m = MyManager()
                m.start()
          
                c0 = m.counter( range(INPUT.count()),INF=0, ifprint = 1,  prefix = '[Worker]',
                              per = bsize,)
                c1 = m.counter( [],INF=1, ifprint = 1, prefix = '[Listener]',
                              step = bsize, per = bsize, )
                fdict = m.dict(self.forward_dict)
                q = m.Queue();          

                if 1:
                    pool = mp.Pool( pcount )
                    watcher  = mp.Process( target = local_listener, args = [OUTPUT], kwargs = {'q':q,'c0':c0,'c1':c1,
                                                                              'fdict':fdict
                                                                                              },)
                    watcher.start()
                    jobs = []

                    for INPUT_curr in INPUTs:
#                         job = pool.Process
                        job  = pool.apply_async(local_worker, (INPUT_curr,), {'q':q, 'field': self.field,'c0':c0,'c1':c1,
                                                                             })
                        jobs.append(job)
                    for job in jobs:
                        job.get()

                    q.put(None)
                    watcher.join()
                    OUTPUT = q.get() 
                pool.close()
                pool.join()
                if not self.force:
                    test__raw( OUTPUT, reverse_dict = self.reverse_dict, letter = self.letter, seqDB_curr = self.sDB)
        # D_raw = OUTPUT
        self.D_raw = OUTPUT

        return OUTPUT
    def Dnorm(self, D_raw = None):
        ##### Calculate D_norm from D_raw

        l = self.l
        OUTPUT = dok_matrix( (l+1,l+1), dtype =  np.float)
#       ########## it's tricky to check whether "None" has been changed to a numpy array.
#         else:
#             raise Exception(' A D_raw must be input! ')
        if isinstance(D_raw,type(None)):
            INPUT = self.D_raw
        else:
            INPUT = D_raw

        it  = using_tocoo_izip( INPUT )
        c=counter(it,per=1000)
        it  = using_tocoo_izip( INPUT )
        
        h1s = []
        h2s = []
        h3s = []
        for x,y,v in it:
            c.count()    
            h1s += [self.hcounts[x]]
            h2s += [self.hcounts[y]]
            h3s += [v]

        BUFlst = ISS_Dnorm_new(
            np.array(h1s),
            np.array(h2s),
            np.array(h3s),
        )


        it  = using_tocoo_izip( INPUT )
        d = dict()
        for (x,y,_),v in izip( it, BUFlst):
            k = (x,y)
            d[k] = v

        OUTPUT.update(d)

        if not self.force:
            test__norm( OUTPUT, self.reverse_dict, self.letter, seqDB_curr = self.sDB,
          norm_func = ISS_Dnorm_new)
            test__key( OUTPUT )
        self.D_norm = OUTPUT
        return OUTPUT
    def Dgeoavg(self, D_raw = None):
        ##### Calculate D_norm from D_raw

        l = self.l
        OUTPUT = dok_matrix( (l+1,l+1), dtype =  np.float)
        
        INPUT = self.D_raw if isinstance(D_raw,type(None)) else D_raw
#         else:
#             INPUT = D_raw

        it  = using_tocoo_izip( INPUT )
        c=counter(it,per=1000)
        it  = using_tocoo_izip( INPUT )
        
        h1s = []
        h2s = []
        h3s = []
        for x,y,v in it:
            c.count()
            h1s += [self.hcounts[x]]
            h2s += [self.hcounts[y]]

        BUFlst = np.sqrt(
            np.multiply( np.array(h1s) , np.array(h2s))
        )


        it  = using_tocoo_izip( INPUT )
        d = dict()
        
        for (x,y,_),v in izip( it, BUFlst):
            k = (x,y)
            d[k] = v

        OUTPUT.update(d)

#         if not self.force:
#             test__norm( OUTPUT, self.reverse_dict, self.letter, seqDB_curr = self.sDB,
#           norm_func = ISS_Dnorm_new)
#             test__key( OUTPUT )
        self.D_geoavg = OUTPUT
        return OUTPUT
    def load(self, name, fields = None, **kwargs):
        fields = fields or ['D_raw','D_norm','hcounts', 'D_both'] ;
        for f in fields:
            fname = '%s/%s' % (f, name, )
            try:
                setattr(self, f, 
                pk_load( fname, **kwargs),
                )
                print "[Msg] managed to load ""%s"" from %s" % (f, fname)
            except:
                print "[Msg] failed  to load ""%s"" from %s" % (f, fname)
    def MySQL_hcount(self):
        def callback(buf):
            hit_summary.objects.bulk_create(buf)
            
        reset_database_connection()
        behave = "inserting hit_summary for CATH nodes "
        if isinstance(self.hcounts, type(None)):
            self.hit_sum()
        nodes = classification.objects.in_bulk(self.reverse_dict).items()
        
        with transaction.atomic():
            buf = []
#             for node in self.reverse_dict:
            c = counter([],INF=1, per = 1E4, prefix = behave)
            for (node_id,node),hcount in zip(nodes,self.hcounts):
                obj  = draft_hsummary( node, self.sDB, hcount)
                if obj:
                    buf.append(obj)
                    c.count()
                if not c.i % 1E4:
        #             hit4cath2cath.objects.bulk_create(buf)
                    callback(buf)
                    buf = []    
            callback(buf)
        c.summary()
        pass
    def Dboth(self):
        if isinstance(self.D_both, type(None) ):
            self.D_both = wrap_mat(
                wrap_mat(self.D_raw,self.D_norm), self.D_geoavg
            )
            pk_dump( ('D_both', self.D_both), self.name )
        return self.D_both
        pass
    def to_MySQL(self):
        reset_database_connection()
        behave = "inserting ISS hits between S35 "
        cutoff = 0.0

        ###################################################
        ##### Inserting S35-S35 ISS-hit ###################
        ###################################################
        ###################################################

        if isinstance(self.D_both, type(None) ):
            self.Dboth()
                
        it0 = self.D_both.todok().iteritems()
        dct_nodes = self.reverse_dict
        # D_curr = D_norm.todok()

        ##############################################
        ##### This need to be set manually ##########
        ############################################
        # dct_nodes = dict(HMMprofile.objects.values_list('id','cath_node'))
        # cutoff = 0.5


        c = counter([],INF=1, per = 1E4, prefix = behave)


        def callback( buf ):
            hit4cath2cath.objects.bulk_create(buf)

        with transaction.atomic():
            buf = []

            for (x,y),((v1,v2),v3) in it0:
                if v2 < cutoff :
                    continue
                node1 = dct_nodes[x]
                node2 = dct_nodes[y]
                nodes = sorted([node1,node2])
        #         hmm1 = (nodes[0])
        #         h
        #         assert nodes[0]
                jdict = {
                    'node1_id': nodes[0],
                    'node2_id': nodes[1],
                    'ISS_raw': v1,
                    'ISS_norm':v2,
                    'hcount_geoavg':v3,
                    'seqDB': self.sDB,
                }

                obj = hit4cath2cath(**jdict)
                buf.append(obj)
                c.count()

                if not c.i % 1000:
        #             hit4cath2cath.objects.bulk_create(buf)
                    callback(buf)
                    buf = []

            hit4cath2cath.objects.bulk_create(buf)
            buf = []
        c.summary()

In [12]:
classification.objects.get(pk = 309887).hit_summary_set.first().hcount

312L

In [ ]:
hit = F('price')
product.save()
print(product.price)            # <CombinedExpression: F(price) + Value(1)>
product.refresh_from_db()
print(product.price)            # Decimal('13.00')

In [15]:
reset_database_connection()
with transaction.atomic():
        behave = "inserting hit_summary for CATH nodes "
        c = counter([],INF=1, per = 1E3, prefix = behave)
        for hit in hit4cath2cath.objects.all().prefetch_related("node1","node2").iterator():
            try:
                c.count()
                
                hit.update_hcount_geoavg()
                hit.save()
            except Exception as e:
                c.fail("",hit)
            finally: 
                c.count()
        c.summary()

inserting hit_summary for CATH nodes 0 of -1
inserting hit_summary for CATH nodes 1000 of -1
inserting hit_summary for CATH nodes 2000 of -1
inserting hit_summary for CATH nodes 3000 of -1
inserting hit_summary for CATH nodes 4000 of -1
inserting hit_summary for CATH nodes 5000 of -1
inserting hit_summary for CATH nodes 6000 of -1
inserting hit_summary for CATH nodes 7000 of -1
inserting hit_summary for CATH nodes 8000 of -1
inserting hit_summary for CATH nodes 9000 of -1
inserting hit_summary for CATH nodes 10000 of -1
inserting hit_summary for CATH nodes 11000 of -1
inserting hit_summary for CATH nodes 12000 of -1
inserting hit_summary for CATH nodes 13000 of -1
inserting hit_summary for CATH nodes 14000 of -1
inserting hit_summary for CATH nodes 15000 of -1
inserting hit_summary for CATH nodes 16000 of -1
inserting hit_summary for CATH nodes 17000 of -1
inserting hit_summary for CATH nodes 18000 of -1
inserting hit_summary for CATH nodes 19000 of -1
inserting hit_summary for CATH no

KeyboardInterrupt: 

In [29]:
hits = hit4cath2cath.objects.all()
# hit = hits.first()
# hits = hits.annotate(hcount = Case(When( node1__hit_summary__seqDB = F("seqDB"), then = F("node1__hit_summary__hcount") ),default = 0))

hits = hits.update(hcount_geoavg = 
                   Case(
                       When( Q(node1__hit_summary__seqDB = F("seqDB")) and Q(node2__hit_summary__seqDB = F("seqDB")), 
                            then = ( F("node1__hit_summary__hcount")*F("node2__hit_summary__hcount")) **0.5,),
                       default = None
                   )
                  )
                   

# hits.update(hcount_geoavg = When())
hits.first().hcount

FieldError: Joined field references are not permitted in this query

In [15]:
# post_data = {'name': 'Gladys'}

post_data = {'name': matObj.hcounts }

response = requests.post('http://example.com', data=post_data)
content = response.content
print content

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 50px;
        background-color: #fff;
        border-radius: 1em;
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        body {
            background-color: #fff;
        }
        div {
            width: auto;
            margin: 0 auto;
            border-radius: 0;
            padding: 1em;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This doma

In [11]:
hit.update_hcount_geoavg()

76.4525996941896

In [ ]:
Entry.objects.all().update(n_pingbacks=F('n_pingbacks') + 1)

In [16]:
print c.e

None


In [66]:
q = matObj.hits.first()

<sequence: 102mA00 from <class 'tst.models.seqDB'>>